In [1]:
import numpy as np
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Parameter, Tensor, ms_function

In [2]:
def make_batch(seq_data, word_dict,vocab_size):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(vocab_size)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [3]:
class TextLSTM(nn.Cell):
    def __init__(self, batch_size, vocab_size, hidden_size):
        super(TextLSTM,self).__init__()
        self.lstm = nn.LSTM(input_size=vocab_size, hidden_size=hidden_size)
        self.W = nn.Dense(hidden_size, vocab_size, has_bias=False)
        self.b = Parameter(Tensor(np.ones(vocab_size), mindspore.float32), 'b')
        
        self.n_steps = n_steps

    def construct(self, X):
        input = X.transpose((1, 0, 2))  
        outputs, (_, _) = self.lstm(input)
        outputs = outputs[-1] 
        model = self.W(outputs) + self.b  
        return model

In [4]:
n_steps = 3 
hidden_size = 128 

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
vocab_size = len(word_dict)  

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

In [5]:
input_batch, target_batch = make_batch(seq_data, word_dict, vocab_size)
input_batch = Tensor(input_batch, mindspore.float32)
target_batch = Tensor(target_batch, mindspore.int32)

In [6]:
batch_size = len(input_batch)

In [7]:
model = TextLSTM(batch_size, vocab_size, hidden_size)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [9]:
def forward(inputs, targets):
    logits = model(inputs)
    loss = criterion(logits, targets)
    return loss

In [10]:
grad_fn = ops.value_and_grad(forward, None, optimizer.parameters)

In [11]:
@ms_function
def train_step(inputs, targets):
    loss, grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss

In [12]:
model.set_train()
# Training
epoch = 1000
for step in range(epoch):
    loss = train_step(input_batch, target_batch)
    if (step + 1) % 100 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost = ', '{:.6f}'.format(loss.asnumpy()))

Epoch: 0100 cost =  1.123684
Epoch: 0200 cost =  0.125290
Epoch: 0300 cost =  0.027129
Epoch: 0400 cost =  0.010317
Epoch: 0500 cost =  0.005415
Epoch: 0600 cost =  0.003387
Epoch: 0700 cost =  0.002342
Epoch: 0800 cost =  0.001727
Epoch: 0900 cost =  0.001330
Epoch: 1000 cost =  0.001058


In [13]:
inputs = [sen[:3] for sen in seq_data]

In [14]:
predict = model(input_batch).asnumpy().argmax(axis=1)
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
